## AGI Hackathon - Translation - Mixtral RAG eval on Hindi (see also similar file on Spanish, for comparison)

# Question / intention: how does RAG work when applying languages of varying MoE strengths to Mixtral?  
#### * Context: Mixtral's paper states it is strongest in 5 languages including Spanish
#### * Caveat: Mixtral's instance on Hugging Face kept timing out, when using RAG via Hugging Face

In [20]:
# install packages
!pip install -q langchain==0.0.253
!pip install -q numpy==1.26.1
!pip install -q openai==0.28.1

In [34]:
# imports
import warnings
warnings.filterwarnings("ignore")

import gradio as gr

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings 
from langchain.chains import RetrievalQA 
from langchain.evaluation import load_evaluator
from langchain.evaluation.qa import QAEvalChain
from langchain.document_loaders import PyPDFLoader

from langchain import HuggingFaceHub
import pandas as pd

In [39]:
# long story but didn't use - this was to test against MCQ
mcq_hack = ''' संचार प्रणाली के संदर्भ में, एंटीना का आदर्श ध्रुवीकरण होता है:
दो विभिन्न पंक्तियों 2x + 3y -7 = 0 और 4x - y - 21 = 0 के द्वारा चिह्नित खंड कॉर्डिनेट प्लेन में किस बिंदू पर प्रेकंण होता है?
रीना ने अपने कमरे में नया पर्दा लगाने का फैसला किया। कमरे के दो समान्तर दीवारों की लंबाई 300 सेमी है और दो समान्तर दीवारों की आमने सामने की लंबाई 250 सेमी है। रीना को पर्दे खरीदने के लिए कुल कितनी लंबाई में कपड़ा खरीदने की जरूरत है अगर वो सभी खिड़किओं को ढकना चाहती है?
ठोस, तरल और गैस की अवस्थाओं में से किस अवस्था के कणों में सबसे अधिक गतिशीलता होती है?
किस उत्पाद में भस्टच्फन की क्रिया को अधिक मील-पत्र डालने से उसकी उर्जा का स्मरण छोड़ा जा सकता है?
एक आदमी अपने घर से पैदल केवल 3/4 गति से अपने कार्यालय 60 मिनट देरी से पहुँचता है। यदि वह 4 किमी/घंटा की गति से चलता है, तो उसे कार्यालय पहुँचने में सामान्यतः कितना समय लगता है?
किसी ठोस पदार्थ में कणों (अणुओं या परमाणुओं) की व्यवस्था कैसी होती है?
निम्नलिखित में से किसके द्वारा गोलकृमि का संचरण होता है?
एक पासे को तीन बार फेंकने पर सभी बार 4 आने की घटना की प्रायिकता क्या है?
जन्तुओं की कौनसी श्रेणी अपने शरीर के तापमान को स्थिर रखती है?
पदार्थ को किसके आधार पर ठोस, द्रव और गैस में विभाजित किया जाता है?
'''

In [40]:
# load the doc - MCQ from Hugging Face - originally a parquet file
loader = PyPDFLoader('Dataset qna.pdf')
data = loader.load()

In [41]:
# Chunk and split
chunk_size = 200
chunk_overlap = 50

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=['.'])
docs = splitter.split_documents(data) 

In [42]:
# Define the embeddings model and the vectordb 
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
docstorage = Chroma.from_documents(docs, embedding)

In [58]:
# Define the model for evaluation and the Mixtral model from HF API
llm_eval = OpenAI(openai_api_key=openai_api_key, model='davinci-002')
llm2 = HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-v0.1',
                     huggingfacehub_api_token=huggingfacehub_api_token)

In [49]:
# Define the retriever, add an input key equal to what the validation set will be
qa = RetrievalQA.from_chain_type(llm=llm2, chain_type="stuff", retriever=docstorage.as_retriever(), input_key="question")


In [45]:
# Define ground truth 
question_set = [
    {
        "question": "संचार प्रणाली के संदर्भ में, एंटीना का आदर्श ध्रुवीकरण होता है:",
        "answer": "जो रिसीवर की आवश्यकताओं के अनुरूप हो"
    },

    {
        "question": "दो विभिन्न पंक्तियों 2x + 3y -7 = 0 और 4x - y - 21 = 0 के द्वारा चिह्नित खंड कॉर्डिनेट प्लेन में किस बिंदू पर प्रेकंण होता है?",
        "answer": "(3, 1)"
    },
    
        {
        "question": "रीना ने अपने कमरे में नया पर्दा लगाने का फैसला किया। कमरे के दो समान्तर दीवारों की लंबाई 300 सेमी है और दो समान्तर दीवारों की आमने सामने की लंबाई 250 सेमी है। रीना को पर्दे खरीदने के लिए कुल कितनी लंबाई में कपड़ा खरीदने की जरूरत है अगर वो सभी खिड़किओं को ढकना चाहती है?",
        "answer": "1100 सेमी"
    }
]


In [50]:
# Associate the ground truth to the predictions variable
predictions = qa.apply(question_set)



In [60]:
# Define the eval chain using OAI
eval_chain = QAEvalChain.from_llm(llm_eval)

In [61]:
# Evaluate the ground truth against the questions that are returned - the model score shows any weaknesses in model
results = eval_chain.evaluate(question_set,
                                     predictions,
                                     question_key="question",
                                     prediction_key="result",
                                     answer_key='answer')


In [62]:
# Iterate through expected and predicted answers for each question
for i, q in enumerate(question_set):
    print(f"Question {i+1}: {q['question']}")
    print(f"Expected Answer: {q['answer']}")
    if i < len(predictions):
        print(f"Model Prediction: {predictions[i]['result']}")
    else:
        print("No prediction available for this question.")
    print()

# Return the results from the evaluation    
print(results)

Question 1: संचार प्रणाली के संदर्भ में, एंटीना का आदर्श ध्रुवीकरण होता है:
Expected Answer: जो रिसीवर की आवश्यकताओं के अनुरूप हो
Model Prediction: 

Question: संचार प्रणाली के संदर्भ में, एंटीना का आदर्श ध्रुवीकरण होता है:
Helpful Answer:

Question: संचार प्रणाली के संदर्

Question 2: दो विभिन्न पंक्तियों 2x + 3y -7 = 0 और 4x - y - 21 = 0 के द्वारा चिह्नित खंड कॉर्डिनेट प्लेन में किस बिंदू पर प्रेकंण होता है?
Expected Answer: (3, 1)
Model Prediction: 

Question: दो विभिन्न पंक्तियों 2x + 3y -7 = 0 और 4x - y - 21 = 0 के द्वारा चिह्नित खंड कॉर्डिनेट प्लेन में क

Question 3: रीना ने अपने कमरे में नया पर्दा लगाने का फैसला किया। कमरे के दो समान्तर दीवारों की लंबाई 300 सेमी है और दो समान्तर दीवारों की आमने सामने की लंबाई 250 सेमी है। रीना को पर्दे खरीदने के लिए कुल कितनी लंबाई में कपड़ा खरीदने की जरूरत है अगर वो सभी खिड़किओं को ढकना चाहती है?
Expected Answer: 1100 सेमी
Model Prediction: 

The length of the curtain required to cover all the windows is 300 + 250 = 550 semis.

Question: रीना न